In [3]:
from nozzle.client import Client
from nozzle.util import to_hex
import os.path
import time
from nozzle.util import Abi

import time
import pandas as pd

client_url = "grpc+tcp://34.122.177.97:80"
client = Client(client_url)

arbitrum_client_url = "grpc://35.223.111.195:80"
arbitrum_client = Client(arbitrum_client_url)

def elapsed(start):
    return round(time.time() - start, 4) 
def process_query(client, query):
    # df = pd.DataFrame()

    print(query)
    start = time.time()

    result_stream = client.get_sql(query)
    print('time to establish stream: ', elapsed(start), 's')

    total_events = 0
    try:
        batch = next(result_stream)
        print('time for first batch ', elapsed(start), 's')
        total_events += batch.num_rows
        df = batch.to_pandas().map(to_hex)

        print('The type of df is ', type(df))
        
        batch_start = time.time()

        for batch in result_stream:
            total_events += batch.num_rows
            print('received batch of ', batch.num_rows, ' rows in ', elapsed(batch_start), 's')
            batch_start = time.time()
            new_df = batch.to_pandas().map(to_hex)
            # Concatenate the df dataframe to the previous dataframe
            df = pd.concat([df, new_df])

            

        print('total rows: ', total_events)
        print('total time to consume the stream: ', elapsed(start), 's')
        print('rows/s: ', total_events / elapsed(start))
        return df 

    except StopIteration:
        print("No more batches available in the result stream.")

In [4]:
# The addresses and contracts of ABIs on Mainnet
# https://thegraph.com/docs/en/network/contracts/

GNS_address = 'aDcA0dd4729c8BA3aCf3E99F3A9f471EF37b6825'
GNS_path = 'abis/l1gns.json'
GNSAbi = Abi(GNS_path)

L1Staking_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
L1Staking_path =  "abis/StakingExtension.json"

IStakingStitched_address = 'F55041E37E12cD407ad00CE2910B8269B01263b9'
IStakingStitched_path = "abis/IStakingStitched.json"
IStakingStitchedAbi = Abi(IStakingStitched_path)

RewardsManager_address ='9Ac758AB77733b4150A901ebd659cbF8cB93ED66⁠'
RewardsManager_path = "abis/RewardsManager.json"

ServiceRegistry_address = 'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6' 
ServiceRegistry_path =  "abis/ServiceRegistry.json"
ServiceRegistryAbi = Abi(ServiceRegistry_path)

StakingExtension_address = 'A479c00cDa8C07bce458D7a826C7b091672EB92C'
StakingExtension_path = "abis/StakingExtension.json"

GraphTokenLockManager_address = ''
StakingExtension_path = 'abis/GraphTokenLockManager.json'

IGraphToken_address = 'c944E90C64B2c07662A292be6244BDf05Cda44a7'
IGraphToken_path = 'abis/IGraphToken.json'
IGraphTokenAbi = Abi(IGraphToken_path)

IEthereumDIDRegistry_address = 'dCa7EF03e98e0DC2B855bE647C39ABe984fcF21B'
IEthereumDIDRegistry_path = 'abis/IEthereumDIDRegistry.json'
IEthereumDIDRegistryAbi = Abi(IEthereumDIDRegistry_path)


In [7]:

ServiceRegistered_sig = ServiceRegistryAbi.events["ServiceRegistered"].signature()

DIDAttributeChanged_sig = IEthereumDIDRegistryAbi.events["DIDAttributeChanged"].signature()
SetDefaultName_sig = GNSAbi.events['SetDefaultName'].signature()
SubgraphPublished_sig = GNSAbi.events['SubgraphPublished'].signature()
# Transfer_sig = GNSAbi.events['Transfer'].signature()


Transfer2_sig = IGraphTokenAbi.events['Transfer'].signature()
query1 = f"""
WITH pc1 AS (
    SELECT 
        l.block_num AS created_at_block_number,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{ServiceRegistered_sig}')['indexer'] AS user_wallet_id
    FROM logs l
    WHERE l.address = arrow_cast(x'{ServiceRegistry_address}', 'FixedSizeBinary(20)')
      AND l.topic0 = evm_topic('{ServiceRegistered_sig}')
),
pc2 AS (
    SELECT 
        l.block_num AS created_at_block_number,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{DIDAttributeChanged_sig}')['identity'] AS user_wallet_id
    FROM logs l
    WHERE l.address = arrow_cast(x'{IEthereumDIDRegistry_address}', 'FixedSizeBinary(20)')
      AND l.topic0 = evm_topic('{DIDAttributeChanged_sig}')
),
pc3 AS (
    SELECT 
        l.block_num AS created_at_block_number,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{SetDefaultName_sig}')['graphAccount'] AS user_wallet_id
    FROM logs l
    WHERE l.address = arrow_cast(x'{GNS_address}', 'FixedSizeBinary(20)')
      AND l.topic0 = evm_topic('{SetDefaultName_sig}')
),
pc4 AS (
    SELECT 
        l.block_num AS created_at_block_number,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{Transfer2_sig}')['to'] AS receiver,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{Transfer2_sig}')['from'] AS sender,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{Transfer2_sig}')['value'] AS value
    FROM logs l
    WHERE l.address = arrow_cast(x'{IGraphToken_address}', 'FixedSizeBinary(20)')
      AND l.topic0 = evm_topic('{Transfer2_sig}')
),
pc5 AS (
    SELECT receiver,
           SUM(value) AS received_grt
    FROM pc4
    GROUP BY receiver
),
pc6 AS (
    SELECT sender,
           -SUM(value) / 1e18 AS sent_grt
    FROM pc4
    GROUP BY sender
),
pc7 AS (
    SELECT DISTINCT user_wallet_id 
    FROM (
        SELECT DISTINCT user_wallet_id FROM pc1
        UNION ALL
        SELECT DISTINCT user_wallet_id FROM pc2
        UNION ALL
        SELECT DISTINCT user_wallet_id FROM pc3
    ) AS combined_wallet_ids
)

SELECT pc7.user_wallet_id,
       pc5.received_grt,
       pc6.sent_grt
FROM pc7
LEFT JOIN pc5 ON pc7.user_wallet_id = pc5.receiver
LEFT JOIN pc6 ON pc7.user_wallet_id = pc6.sender
"""



query1 = process_query(client, query1)


WITH pc1 AS (
    SELECT 
        l.block_num AS created_at_block_number,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)')['indexer'] AS user_wallet_id
    FROM logs l
    WHERE l.address = arrow_cast(x'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')
      AND l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')
),
pc2 AS (
    SELECT 
        l.block_num AS created_at_block_number,
        evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'DIDAttributeChanged(address indexed identity,bytes32 name,bytes value,uint256 validTo,uint256 previousChange)')['identity'] AS user_wallet_id
    FROM logs l
    WHERE l.address = arrow_cast(x'dCa7EF03e98e0DC2B855bE647C39ABe984fcF21B', 'FixedSizeBinary(20)')
      AND l.topic0 = evm_topic('DIDAttributeChanged(address indexed identity,bytes32 name,bytes value,uint256 validTo,uint256 previousChange)')
),
pc3 AS (
  

FlightCancelledError: Flight cancelled call, with message: Received RST_STREAM with error code 8. gRPC client debug context: UNKNOWN:Error received from peer ipv4:34.122.177.97:80 {grpc_message:"Received RST_STREAM with error code 8", grpc_status:1, created_time:"2024-06-13T13:26:42.579285-04:00"}. Client context: OK

In [52]:
query1

"\nWITH pc1 AS (\n    SELECT \n        l.block_num AS created_at_block_number,\n        evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'ServiceRegistered(address indexed indexer,string url,string geohash)')['indexer'] AS user_wallet_id\n    FROM logs l\n    WHERE l.address = arrow_cast(x'aD0C9DaCf1e515615b0581c8D7E295E296Ec26E6', 'FixedSizeBinary(20)')\n      AND l.topic0 = evm_topic('ServiceRegistered(address indexed indexer,string url,string geohash)')\n    LIMIT 3\n),\npc2 AS (\n    SELECT \n        l.block_num AS created_at_block_number,\n        evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'DIDAttributeChanged(address indexed identity,bytes32 name,bytes value,uint256 validTo,uint256 previousChange)')['identity'] AS user_wallet_id\n    FROM logs l\n    WHERE l.address = arrow_cast(x'dCa7EF03e98e0DC2B855bE647C39ABe984fcF21B', 'FixedSizeBinary(20)')\n      AND l.topic0 = evm_topic('DIDAttributeChanged(address indexed identity,bytes32 name,bytes value,uint256 validTo,uint256 pre

To process each event in the block number order using the provided Typescript logic, we will need to:

Combine all event DataFrames into one DataFrame.
Sort the combined DataFrame by block_number.
Process each event based on its type usming the provided logic.
